<a href="https://colab.research.google.com/github/kynk94/11th_Project/blob/master/dance/preprocess_by_%EC%84%9C%EC%84%9D%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocess for Everybody dance now - by 서석현

1. preparation   
  * 영상 2개(target, source)
  * target이 바꾸고 싶은 영상(ex 내 모습)이라고 생각하면 되고 이 친구를 학습시켜야함
  * source는 춤추는 영상 
  * Realtime, Multi person pose estimation에 대한 파일이 필요 - git clone으로 극복  
  
2. summary of the code
  * google drive 마운트 시키기! (쉽게 말하면 colab이랑 구글 드라이브랑 연동하기)
  * 우선 동영상 다운 받기 - colab에서 잘 안돌아가서 로컬에서 다운 받기 추천
  * cv2로 동영상을 읽어서 프레임으로 쪼개기 - 주로 1초에 20~30 프레임 정도 나옴
  * png 파일에 대해서 pose -estimation하기
  
3. Recommendation  
  * 로컬이 아닌 colab에서 돌리길 추천합니다.
  * 컴퓨터가 좋으면 사실 상관 없습니다. 그냥 돌려도 괜찮습니다.

In [0]:
# 구글 드라이브 연동 시키기 -> 링크에 나와 있는 친구 복사해서 붙여넣기 하면 됨
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
import cv2
#from pytube import YouTube as youtube #유투브에서 동영상 다운받기
from pathlib import Path

drive = Path('/gdrive/My Drive/source/target') # 제 구글 드라이브에 폴더 만드는 작업입니다.
drive.mkdir(exist_ok=True)
images = drive.joinpath('images')
images.mkdir(exist_ok=True)

#yt = youtube('https://www.youtube.com/watch?v=4wpn3y66mzY') # 이 친구는 로컬에서 돌리는 걸 추천
#yt.streams.first().download(drive, 'mv') # mv.mp4로 이름 지어줍니다.

cap = cv2.VideoCapture(str(drive.joinpath('mv-2.mp4'))) # 이건 이미 제가 받은 친구들 프레임으로 자르는 과정입니다.
i = 0
while(cap.isOpened()): # 열려 있을 때까지
    flag, frame = cap.read() # 동영상을 읽고 (읽는 방식이 프레임로 읽는다고 생각하면 편합니다.)
    if not(flag) or i == 1000: # 1000까지만 읽음 -> 늘려도 괜찮
        break
    cv2.imwrite(str(images.joinpath(f'img_{i:04d}.png')), frame) #img_0001.png로 저장
    i += 1



### Pose Estimation from git clone

이 친구를 사용해서 사진 하나에서 사람이 어떤 포즈를 하고 있는지 묘사

 * pretrained data needed - check the file pose_model.pth
 * 팀 깃허브에 다운로드 링크 공유해둠


In [0]:
!git clone https://github.com/tensorboy/pytorch_Realtime_Multi-Person_Pose_Estimation.git

In [0]:
cd pytorch_Realtime_Multi-Person_Pose_Estimation

/content/pytorch_Realtime_Multi-Person_Pose_Estimation


In [0]:
from network.rtpose_vgg import get_model # pose-estimation에서 옴
from evaluate.coco_eval import get_multiplier, get_outputs # pose-estimation에서 옴
import numpy as np
import math
import cv2
from skimage import filters
from scipy import ndimage
import os

아래 코드는 다음과 같은 주소에서 따왔습니다.

각 함수에 대해서 간략하게 소개를 하자면 

1. remove_noise:   
input : 이미지  
output: argmax를 통한 나머지 흑백 처리 -> 예전에 제가 올려드린 그 검은색 사진 생각하시면 될거 같습니다.  

2. 라벨 만들기 -> 이 부분은 어려운데 걍 뼈다귀 만드는 과정인거 같긴합니다. 정확히 표현하면 뼈다귀 joint 점을 찾는 과정인거 같아용

3. get_pose : 검은색 사진에 흰색 뼈다귀가 나온다고 생각하시면 편합니다.


In [0]:
# openpose reference https://github.com/CUHKSZ-TQL/EverybodyDanceNow_reproduce_pytorch/src

from network.post import * #pose-estimation에서 옴

def remove_noise(img):
    th = filters.threshold_otsu(img)
    bin_img = img > th
    regions, num = ndimage.label(bin_img)
    areas = []
    for i in range(num):
        areas.append(np.sum(regions == i+1))
    img[regions != np.argmax(areas)+1] = 0
    return img


def create_label(shape, joint_list, person_to_joint_assoc):
    label = np.zeros(shape, dtype=np.uint8)
    cord_list = []
    for limb_type in range(17):
        for person_joint_info in person_to_joint_assoc:
            joint_indices = person_joint_info[joint_to_limb_heatmap_relationship[limb_type]].astype(int)
            if -1 in joint_indices:
                continue
            joint_coords = joint_list[joint_indices, :2]
            coords_center = tuple(np.round(np.mean(joint_coords, 0)).astype(int))
            cord_list.append(joint_coords[0])
            limb_dir = joint_coords[0, :] - joint_coords[1, :]
            limb_length = np.linalg.norm(limb_dir)
            angle = math.degrees(math.atan2(limb_dir[1], limb_dir[0]))
            polygon = cv2.ellipse2Poly(coords_center, (int(limb_length / 2), 4), int(angle), 0, 360, 1)
            cv2.fillConvexPoly(label, polygon, limb_type+1)
    return label,cord_list


def get_pose(param, heatmaps, pafs):
    shape = heatmaps.shape[:2]
    # Bottom-up approach:
    # Step 1: find all joints in the image (organized by joint type: [0]=nose,
    # [1]=neck...)
    joint_list_per_joint_type = NMS(param, heatmaps)
    # joint_list is an unravel'd version of joint_list_per_joint, where we add
    # a 5th column to indicate the joint_type (0=nose, 1=neck...)
    joint_list = np.array([tuple(peak) + (joint_type,) for joint_type,
                           joint_peaks in enumerate(joint_list_per_joint_type) for peak in joint_peaks])

    # Step 2: find which joints go together to form limbs (which wrists go
    # with which elbows)
    paf_upsamp = cv2.resize(pafs, shape, interpolation=cv2.INTER_CUBIC)
    connected_limbs = find_connected_joints(param, paf_upsamp, joint_list_per_joint_type)

    # Step 3: associate limbs that belong to the same person
    person_to_joint_assoc = group_limbs_of_same_person(connected_limbs, joint_list)

    # (Step 4): plot results
    label,cord_list = create_label(shape, joint_list, person_to_joint_assoc)

    return label, cord_list

In [0]:
# 저장되어 있는 pose-estimation 모델 불러와서 cuda 친구로 보내주기
import torch
weight_name = '/gdrive/My Drive/source/pose_model.pth'
model = get_model('vgg19')     
model.load_state_dict(torch.load(weight_name))
model = torch.nn.DataParallel(model).cuda()
model.float()
model.eval()
pass

Bulding VGG19


In [0]:
#제가 아까 저장했던 사진들에 대한 경로를 지정해줍시다

save_dir = Path('/gdrive/My Drive/source/target')
img_dir = Path('/gdrive/My Drive/source/target/images')
'''make label images for pix2pix'''
#test_img는 원본 파일, #test_label_ori는 포즈가 나오는 검은 사진, # test_head_ori는 머리부분만 추출 -face gan에 사용 -> 논문 참조
test_img_dir = save_dir.joinpath('test_img')
test_img_dir.mkdir(exist_ok=True)
test_label_dir = save_dir.joinpath('test_label_ori')
test_label_dir.mkdir(exist_ok=True)
test_head_dir = save_dir.joinpath('test_head_ori')
test_head_dir.mkdir(exist_ok=True)


In [0]:
# 자 사진을 받아서 포즈를 만들어봅시다.
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
pose_cords = []
for idx in tqdm_notebook(range(len(os.listdir(str(img_dir))))):
    img_path = img_dir.joinpath('img_{:04}.png'.format(idx)) # 아까 저장된 사진을 읽어봅시다.
    img = cv2.imread(str(img_path))
    shape_dst = np.min(img.shape[:2])
    oh = (img.shape[0] - shape_dst) // 2
    ow = (img.shape[1] - shape_dst) // 2

    img = img[oh:oh + shape_dst, ow:ow + shape_dst]
    img = cv2.resize(img, (512, 512)) # resize
    multiplier = get_multiplier(img)
    with torch.no_grad():
        paf, heatmap = get_outputs(multiplier, img, model, 'rtpose') # estimate 모델을 통해서 국수 뽑듯이 뽑음
    r_heatmap = np.array([remove_noise(ht)
                          for ht in heatmap.transpose(2, 0, 1)[:-1]]) \
        .transpose(1, 2, 0)
    heatmap[:, :, :-1] = r_heatmap
    param = {'thre1': 0.1, 'thre2': 0.05, 'thre3': 0.5}
    label, cord = get_pose(param, heatmap, paf)
    index = 13
    crop_size = 25
    try:
        head_cord = cord[index]
    except:
        head_cord = pose_cords[-1] # if there is not head point in picture, use last frame

    pose_cords.append(head_cord)
    head = img[int(head_cord[1] - crop_size): int(head_cord[1] + crop_size),
           int(head_cord[0] - crop_size): int(head_cord[0] + crop_size), :]
    plt.imshow(head)
    plt.savefig(str(test_head_dir.joinpath('pose_{}.jpg'.format(idx))))
    plt.clf()
    cv2.imwrite(str(test_img_dir.joinpath('{:05}.png'.format(idx))), img) # 저장은 00001.png로 
    cv2.imwrite(str(test_label_dir.joinpath('{:05}.png'.format(idx))), label)
    if idx % 100 == 0 and idx != 0:
        pose_cords_arr = np.array(pose_cords, dtype=np.int)
        np.save(str((save_dir.joinpath('pose_source.npy'))), pose_cords_arr) # 1000장에 대한 포즈 정보를 저장하는 것 같습니다.

pose_cords_arr = np.array(pose_cords, dtype=np.int)
np.save(str((save_dir.joinpath('pose_target.npy'))), pose_cords_arr)
torch.cuda.empty_cache()

위 코드를 돌리면 1000장에 약 1시간 걸립니다..

실험 환경

Tesla T4

1000장

자 이제 우리는 학습으로 넘어갑시다